
# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

国勢調査の小地域集計データ（人口構成、労働力構成、住宅形態など）から、その地域の収入を推定することが考えられます。推定のための機械学習・深層学習モデルを構築しなさい。


- データを観察・理解する上で、データの構造を説明しながら、適切なデータ整形を行いなさい
- データ構造や分析結果に対して、少なくとも二つの図で可視化を行いなさい
- モデルの精度を評価し、できるだけ精度が高いモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめなさい


In [ ]:
# ======== ライブラリ読み込み ========
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from google.colab import drive

# ======== データ読み込み ========
age_df = pd.read_csv("/h27_age_df.csv")
gender_df = pd.read_csv("/h27_gender_df2.csv")
job_df = pd.read_csv("/h27_job_df.csv")
work_status_df = pd.read_csv("/h27_work_status_df.csv")
income_district_df = pd.read_csv("/income_district.csv")

# ======== 不要列削除とdistrict_idの統一 ========
def reduce_columns(df):
    return df.drop(columns=["district2_id", "level_identifier", "state_name", "city_name", 
                            "district_name", "district2_name"], errors='ignore')

age_df = reduce_columns(age_df)
gender_df = reduce_columns(gender_df)
job_df = reduce_columns(job_df)
work_status_df = reduce_columns(work_status_df)

# area_code → district_idに変更
income_district_df.rename(columns={'area_code': 'district_id'}, inplace=True)

# 欠損値等の処理
for df in [age_df, gender_df, job_df, work_status_df]:
    df.replace(["x", "-"], pd.NA, inplace=True)
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(inplace=True)

# ======== district_idごとに平均値を取って集約 ========
age_df_grouped = age_df.groupby('district_id').mean().reset_index()
gender_df_grouped = gender_df.groupby('district_id').mean().reset_index()
job_df_grouped = job_df.groupby('district_id').mean().reset_index()
work_status_df_grouped = work_status_df.groupby('district_id').mean().reset_index()

# ======== データ結合 ========
dfs = [age_df_grouped, gender_df_grouped, job_df_grouped, work_status_df_grouped, income_district_df]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='district_id', how='inner')

print("merged_df shape:", merged_df.shape)

# ======== 新たな特徴量作成 ========
# 就業者数合計（男女別）
merged_df['total_employed_male'] = merged_df[['employer_male', 'self_employed_male', 'family_work_male', 'unkonwn_status_male']].sum(axis=1)
merged_df['total_employed_female'] = merged_df[['employer_female', 'self_employed_female', 'family_work_female', 'unkonwn_status_female']].sum(axis=1)
merged_df['total_employed'] = merged_df['total_employed_male'] + merged_df['total_employed_female']

# 男女就業比率
merged_df['male_ratio'] = merged_df['total_employed_male'] / merged_df['total_employed']

# 年齢層人口合計（男女合計）
age_cols = [col for col in merged_df.columns if ('_male' in col or '_female' in col) and col != 'male_ratio']
merged_df['total_population'] = merged_df[age_cols].sum(axis=1)

# 男女人口合計
merged_df['total_male'] = merged_df[[col for col in merged_df.columns if '_male' in col]].sum(axis=1)
merged_df['total_female'] = merged_df[[col for col in merged_df.columns if '_female' in col]].sum(axis=1)

# 男女比（男性比率）
merged_df['male_population_ratio'] = merged_df['total_male'] / (merged_df['total_male'] + merged_df['total_female'])

# 就業者率
work_cols = ['employer_male', 'self_employed_male', 'family_work_male', 'unkonwn_status_male',
             'employer_female', 'self_employed_female', 'family_work_female', 'unkonwn_status_female']
merged_df['total_workers'] = merged_df[work_cols].sum(axis=1)
merged_df['worker_ratio'] = merged_df['total_workers'] / merged_df['total_population']

# 例：職種別男性割合（管理職男性割合）
merged_df['admin_male_ratio'] = merged_df['A_administrative_male'] / merged_df['total_male']


merged_df shape: (1152, 99)

In [ ]:
# ======== データの可視化（図1〜図5） ========

from matplotlib import font_manager

# Google Drive上の日本語フォントのパス
ttf_path = '/content/drive/MyDrive/Noto_Sans_JP-2025/Noto_Sans_JP/NotoSansJP-VariableFont_wght.ttf'

# フォントを読み込み、matplotlibに登録
font_manager.fontManager.addfont(ttf_path)
custom_font = font_manager.FontProperties(fname=ttf_path)
plt.rcParams['font.family'] = custom_font.get_name()

# 以下を追加（LaTeX系のエラー対策）
plt.rcParams['text.usetex'] = False
plt.rcParams['axes.unicode_minus'] = False  # マイナス記号の文字化け防止


In [ ]:
plt.figure(figsize=(10,6))

# x軸に「全体」というラベルをつけて、箱ひげ図を描画
sns.boxplot(x=['全体'] * len(merged_df), y=merged_df['income_mean'])

# タイトルと軸ラベルの設定
plt.suptitle('図1', x=0.0, ha='left', fontsize=12)
plt.title('地域ごとの平均所得の分布')
plt.xlabel('地域カテゴリ')
plt.ylabel('平均所得')

plt.show()


In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(merged_df['total_population'], bins=30, kde=True)
plt.suptitle('図2', x=0.0, ha='left', fontsize=12)
plt.title('地域ごとの総人口の分布')
plt.xlabel('総人口')
plt.ylabel('地域数')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(merged_df['male_population_ratio'], bins=30, kde=True)
plt.suptitle('図3', x=0.0, ha='left', fontsize=12)
plt.title('地域ごとの男性人口比率の分布')
plt.xlabel('男性人口比率')
plt.ylabel('地域数')
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x='total_population', y='income_mean', data=merged_df)
plt.suptitle('図4', x=0.0, ha='left', fontsize=12)
plt.title('総人口と平均所得の関係')
plt.xlabel('総人口')
plt.ylabel('平均所得')
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x='worker_ratio', y='income_mean', data=merged_df)
plt.suptitle('図5', x=0.0, ha='left', fontsize=12)
plt.title('就業率と平均所得の関係')
plt.xlabel('就業率 (worker_ratio)')
plt.ylabel('平均所得')
plt.show()


### データ構造の説明

本分析で用いたデータは、日本の国勢調査の小地域集計データと、同地域の平均収入データである。  
各小地域は一意の識別子 `district_id` によって識別されている。この `district_id` は、市区町村のさらに細かい単位の地域を表し、地域ごとの人口構成や労働力状況を把握するための基本単位である。

### 主要な特徴量の説明

- **人口構成**：年齢層別・男女別の人口数を示す。例えば、`0-4_male` は0〜4歳の男性人口、`100_female` は100歳以上の女性人口を表す。  
- **労働力構成**：就業者数や雇用形態別（雇用者、自営業者、家族従業者など）、男女別の労働者数が含まれる。例えば、`employer_male` は男性の雇用者数。  
- **特徴量エンジニアリングで追加した指標**  
  - `total_employed_male`・`total_employed_female`：男女別の総就業者数  
  - `male_ratio`：就業者に占める男性の割合  
  - `total_population`：年齢層別人口を合計した地域の総人口  
  - `worker_ratio`：総人口に対する就業者の割合（就業率の一種）  
  - `admin_male_ratio`：男性の管理職に該当する人口の割合  

これらの特徴量は地域ごとの社会経済状況を反映し、収入推定に寄与すると考えられる。


### 地域別平均所得のばらつき

箱ひげ図（図1）より、所得のばらつきが大きく、多くの地域の平均所得が450万円付近に集中していることがわかる。また、第1四分位数・第3四分位数ともに、同じくらい平均から離れていることがわかる。

### 総人口や男女比の地域差

ヒストグラム（図2, 図3）では、総人口や男性人口比率の分布を示した。  
総人口は多くの地域で25000人付近に集中していることがわかる。一方、いくつかの地域では非常に大規模な地域も存在する。  
男女比率も地域により多少異なり、人口構成の違いが地域間格差の要因の一つとなる可能性がある。

### 収入と人口構成の関係

散布図（図4, 図5）から、総人口や就業率と平均所得にはそこまではっきり目立った相関は見られない。しかし、総人口と平均所得について、70000人を超える地域になると、平均所得は正の関連があるようにみえる。

In [ ]:
# ======== 最初のモデルで使う特徴量 ========
features = ['total_employed_male', 'total_employed_female', 'male_ratio', 'total_employed']

X = merged_df[features]
y = merged_df['income_mean']

# ======== モデル構築・評価 ========
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("=== 初回モデル結果 ===")
print(f'R² score: {r2_score(y_test, y_pred):.3f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.3f}')

# ======== 特徴量重要度の可視化 ========
importances = model.feature_importances_
feat_imp_df = pd.DataFrame({'feature': features, 'importance': importances}).sort_values(by='importance', ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x='importance', y='feature', data=feat_imp_df, color='skyblue')
plt.title('Feature Importance (初回モデル)')
plt.show()

# ======== 実測値 vs 予測値の散布図 ========
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Income')
plt.ylabel('Predicted Income')
plt.title('Actual vs Predicted Income (初回モデル)')
plt.show()

# ======== モデル改善に向けた特徴量追加 ========
# 先ほど作った年齢人口合計や職種割合などを加えてみる
features_improved = features + [
    'total_population', 'male_population_ratio', 'worker_ratio', 'admin_male_ratio'
]

X2 = merged_df[features_improved]
y2 = y

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model2 = RandomForestRegressor(random_state=42)
model2.fit(X2_train, y2_train)

y2_pred = model2.predict(X2_test)

print("=== 改善モデル結果 ===")
print(f'R² score: {r2_score(y2_test, y2_pred):.3f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y2_test, y2_pred)):.3f}')

# 特徴量重要度の可視化（改善モデル）
importances2 = model2.feature_importances_
feat_imp_df2 = pd.DataFrame({'feature': features_improved, 'importance': importances2}).sort_values(by='importance', ascending=False)

plt.figure(figsize=(8,5))
sns.barplot(x='importance', y='feature', data=feat_imp_df2, color='lightgreen')
plt.title('Feature Importance (改善モデル)')
plt.show()

# 実測 vs 予測散布図（改善モデル）
plt.figure(figsize=(6,6))
plt.scatter(y2_test, y2_pred, alpha=0.6, color='green')
plt.plot([y2_test.min(), y2_test.max()], [y2_test.min(), y2_test.max()], 'r--')
plt.xlabel('Actual Income')
plt.ylabel('Predicted Income')
plt.title('Actual vs Predicted Income (改善モデル)')
plt.show()


=== 初回モデル結果 ===
R² score: 0.166
RMSE: 57.560
...
=== 改善モデル結果 ===
R² score: 0.367
RMSE: 50.155